In this notebook, we simulate the field created by the TRIF electromagnet and save it as a data set. This data set is going to be used to test our calibration approach.

The electromagnet has three poles, that we model as three magnetic dipoles oriented in the same direction (physically, each pole is a coil wrapped around a ferromagnetic core). The dipoles are located in the same plain at the same distance from the axis of symmetry, and their position vectors are at $120^0$ with respect to each other. We also assume that there can be a uniform external magnetic field.

***

## Setup

In [22]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [23]:
# Functions for the field calculations

def mag(x):
    """Calculates magnitude of vector x"""
    return np.sqrt(np.dot(x,x))

def dipole_field(r, r0, m, a):
    """
    Calculates magnetic field of a single dipole
    r - location of the field
    r0 - position of the dipole
    m - magnitude of the dipole. Negative values correspond to the opposite orientation of  the dipole axis
    a - dipole axis"""
    M = m*a   # Magnetic moment vector
    R = r-r0
    if np.dot(R,R) == 0: #This one is to avoid singularities
        return np.array([0, 0, 0])
    return mu0*(3*R*(np.dot(M,R))/(mag(R)**5) - M/(mag(R)**3))

def total_field(r, m_array: np.array, B0: np.array, dipole_axis: np.array) -> np.array:
    """Calculates the total field at point r
    To simplify the data set generation later, the magnitudes of the three dipoles are placed in array m.
    B0 is the external field.
    dipole_axis is the dipole orientation (same for all three)"""
    B1 = dipole_field(r, R1, m_array[0], dipole_axis)
    B2 = dipole_field(r, R2, m_array[1], dipole_axis)
    B3 = dipole_field(r, R3, m_array[2], dipole_axis)
    return B1+B2+B3+B0

In [24]:
# Constants and the setup parameters

mu0 = 10**-7# magnetic constant
m = 1e8 # Scaling factor magnetic dipole magnitude

# Magnetic dipole position vectors
R1 = np.array([1, 0, 1])
R2 = np.array([-np.cos(np.pi/3), np.sin(np.pi/3), 1])
R3 = np.array([-np.cos(np.pi/3), -np.sin(np.pi/3), 1])

# Magnetic dipole orientation (same for all three)
A = np.array([0, 0, 1]) # It is a global variable

# External magnetic field
B_uniform = np.array([0.,0.,0.])

# The magnetometer location. Magnetic field is calculated here
r0 = np.array([0., 0., 0.])

## Generate the calibration data set

Here we create a set of various dipole magnitudes and calculate the corresponding magnetic field.

In the experiment each dipole magnitude corresponds to a voltage applied to the corresponding coil inside the magnet. In case of linear relationship between the voltage and magnetic field vector, it is enough to collect and combine the field-vs-voltage characteristic from each individual coil. We'll save this data set separately as linear_calibration_set.csv.

In the non-linear case, in order to properly train an artificial neural network, we'll additionally need data with all coils receiving voltage simultaneously. In our model, the field-voltage relationship is linear, so we'll add non-linearity but hand later in the corresponding program.

### Individual coils

In [25]:
# Apply voltage to a single coil in a linear fashion

N1 = 501 # Number of training samples for each dipole

V1 = np.concatenate((np.linspace(-10, 10, N1), np.zeros(N1), np.zeros(N1)))

V2 = np.concatenate((np.zeros(N1), np.linspace(-10, 10, N1), np.zeros(N1)))

V3 = np.concatenate((np.zeros(N1), np.zeros(N1), np.linspace(-10, 10, N1)))

V_ind = np.stack((V1, V2, V3), axis=1) # Stack vertically
print("V_ind \n=", V_ind)

V_ind 
= [[-10.     0.     0.  ]
 [ -9.96   0.     0.  ]
 [ -9.92   0.     0.  ]
 ...
 [  0.     0.     9.92]
 [  0.     0.     9.96]
 [  0.     0.    10.  ]]


In [43]:
# Calculate magnetic field

B_ind = []
for V in V_ind:
    B_ind.append(total_field(r0, V * m, B_uniform, A))

individual_calibration_set = pd.DataFrame(np.concatenate((B_ind, V_ind), axis = 1), columns = ["B_x", "B_y", "B_z", "M_1", "M_2", "M_3"]) # Combine fields and voltages into a single dataset

In [44]:
individual_calibration_set.head()

,B_x,B_y,B_z,M_1,M_2,M_3
0,-53.033009,0.0,-17.677670,-10.00,0.0,0.0
1,-52.820877,0.0,-17.606959,-9.96,0.0,0.0
2,-52.608745,0.0,-17.536248,-9.92,0.0,0.0
3,-52.396612,0.0,-17.465537,-9.88,0.0,0.0
4,-52.184480,0.0,-17.394827,-9.84,0.0,0.0


In [45]:
# Save
individual_calibration_set.to_csv('data/individual_calibration_set.csv')

### Full set

In [50]:
# Apply random voltages to all three coils

N2 = 1000 # Total number of values
V_rand = np.random.uniform(-10, 10, (N2, 3)) # Random voltages between -10 and 10
V_full = np.concatenate((V_ind, V_rand)) # stack with the individual voltage set

In [51]:
# Calculate magnetic field

B_full = []
for V in V_full:
    B_full.append(total_field(r0, V * m, B_uniform, A))

full_calibration_set = pd.DataFrame(np.concatenate((B_full, V_full), axis = 1), columns = ["B_x", "B_y", "B_z", "M_1", "M_2", "M_3"]) # Combine fields and voltages into a single dataset

In [54]:
print(full_calibration_set.head())
print(full_calibration_set.tail())

         B_x  B_y        B_z    M_1  M_2  M_3
0 -53.033009  0.0 -17.677670 -10.00  0.0  0.0
1 -52.820877  0.0 -17.606959  -9.96  0.0  0.0
2 -52.608745  0.0 -17.536248  -9.92  0.0  0.0
3 -52.396612  0.0 -17.465537  -9.88  0.0  0.0
4 -52.184480  0.0 -17.394827  -9.84  0.0  0.0
            B_x        B_y        B_z       M_1       M_2       M_3
2498 -16.399891   0.971828 -16.789139 -5.227386 -2.029194 -2.240792
2499  20.601604  38.278116  12.824668  5.008026  5.290544 -3.043843
2500  21.699817 -45.986098 -15.886148 -0.267683 -9.365772  0.646893
2501  42.172734 -20.125074 -13.174852  2.817171 -7.325937 -2.944056
2502   4.910961   4.469027 -38.863646 -6.710853 -7.150347 -8.123399


In [55]:
# Save
full_calibration_set.to_csv('data/full_calibration_set.csv')

### Validation set for deep learning

We'll use this set to monitor the neural network performance during training in the non-linear case.

In [60]:
N_val = 100

V_val = np.random.uniform(-10, 10, (N_val, 3)) # Random voltages between -10 and 10
B_val = []

for V in V_val:
    B_val.append(total_field(r0, V * m, B_uniform, A))

val_set = pd.DataFrame(np.concatenate((B_val, V_val), axis = 1), columns = ["B_x", "B_y", "B_z", "M_1", "M_2", "M_3"])

In [58]:
val_set.head()

,B_x,B_y,B_z,M_1,M_2,M_3
0,23.195081,0.213975,-32.308176,-3.176284,-7.526696,-7.573285
1,-19.909095,-23.740900,6.907727,-1.200196,-0.030683,5.138480
2,-26.053097,-16.836653,-20.293293,-7.101619,-4.021943,-0.356058
3,-20.342484,64.984770,-11.184787,-4.666234,6.244227,-7.905064
4,29.274393,26.067684,-10.781216,1.647096,-1.035046,-6.710826


In [59]:
# Save
val_set.to_csv('data/validation_set.csv')